In [2]:
# If you're in Colab / fresh env:
# !pip -q install pulp

import pulp as pl


In [4]:
# Decision variables: x_c, x_e, x_p
c = {"xc": 50, "xe": 30, "xp": 40}

# Constraints (<= form) + two extra bounds written as constraints
# y1: 2 xc + 1 xe + 1.5 xp <= 100
# y2: 1 xc + 2 xe + 1   xp <= 150
# y3: 3 xc + 1 xe + 2   xp <= 180
# y4: -xc <= -20   (i.e., xc >= 20)
# y5:  xe <= 60


In [5]:
primal = pl.LpProblem("Primal", pl.LpMaximize)

xc = pl.LpVariable("xc", lowBound=0)
xe = pl.LpVariable("xe", lowBound=0)
xp = pl.LpVariable("xp", lowBound=0)

# Objective
primal += 50*xc + 30*xe + 40*xp, "obj"

# Constraints (name them so we can read slacks later)
primal += 2*xc + 1*xe + 1.5*xp <= 100, "y1"
primal += 1*xc + 2*xe + 1*xp   <= 150, "y2"
primal += 3*xc + 1*xe + 2*xp   <= 180, "y3"
primal += -1*xc                <= -20, "y4"   # xc >= 20
primal += 1*xe                 <= 60,  "y5"   # xe <= 60

primal.solve(pl.PULP_CBC_CMD(msg=False))

primal_status = pl.LpStatus[primal.status]
primal_obj = pl.value(primal.objective)
(primal_status, primal_obj)


('Optimal', 2800.0)

In [6]:
sol_primal = {v.name: v.value() for v in [xc, xe, xp]}

slacks = {}
for name, con in primal.constraints.items():
    # PuLP stores constraints in a standardized form; slack() gives slack for <= constraints
    slacks[name] = con.slack

sol_primal, slacks


({'xc': 20.0, 'xe': 60.0, 'xp': 0.0},
 {'y1': -0.0, 'y2': 10.0, 'y3': 60.0, 'y4': -0.0, 'y5': -0.0})

In [7]:
dual = pl.LpProblem("Dual", pl.LpMinimize)

y1 = pl.LpVariable("y1", lowBound=0)
y2 = pl.LpVariable("y2", lowBound=0)
y3 = pl.LpVariable("y3", lowBound=0)
y4 = pl.LpVariable("y4", lowBound=0)
y5 = pl.LpVariable("y5", lowBound=0)

# Dual objective (from the board)
dual += 100*y1 + 150*y2 + 180*y3 - 20*y4 + 60*y5, "obj"

# Dual constraints (A^T y >= c)
dual += 2*y1 + 1*y2 + 3*y3 - 1*y4          >= 50, "xc"
dual += 1*y1 + 2*y2 + 1*y3        + 1*y5   >= 30, "xe"
dual += 1.5*y1 + 1*y2 + 2*y3               >= 40, "xp"

dual.solve(pl.PULP_CBC_CMD(msg=False))

dual_status = pl.LpStatus[dual.status]
dual_obj = pl.value(dual.objective)
(dual_status, dual_obj)


('Optimal', 2800.0)

In [8]:
sol_dual = {v.name: v.value() for v in [y1, y2, y3, y4, y5]}

gap = abs(primal_obj - dual_obj)

sol_dual, {"primal_obj": primal_obj, "dual_obj": dual_obj, "abs_gap": gap}


({'y1': 30.0, 'y2': 0.0, 'y3': 0.0, 'y4': 10.0, 'y5': 0.0},
 {'primal_obj': 2800.0, 'dual_obj': 2800.0, 'abs_gap': 0.0})

In [9]:
# For primal constraints (<=): y_i * slack_i should be ~ 0 at optimum
cs_primal = {name: sol_dual[name]*slacks[name] for name in ["y1","y2","y3","y4","y5"]}

# For dual constraints (>=): (A^T y - c)_j * x_j should be ~ 0
dual_con_slack = {name: dual.constraints[name].slack for name in ["xc","xe","xp"]}
cs_dual = {
    "xc": dual_con_slack["xc"] * sol_primal["xc"],
    "xe": dual_con_slack["xe"] * sol_primal["xe"],
    "xp": dual_con_slack["xp"] * sol_primal["xp"],
}

cs_primal, dual_con_slack, cs_dual


({'y1': -0.0, 'y2': 0.0, 'y3': 0.0, 'y4': -0.0, 'y5': -0.0},
 {'xc': -0.0, 'xe': -0.0, 'xp': -5.0},
 {'xc': -0.0, 'xe': -0.0, 'xp': -0.0})